In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [6]:
#Gpu tanımla 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Hiperparametreleri tanımla 
num_epochs = 10
batch_size = 64
learning_rate = 0.001
trainset = torchvision.datasets.MNIST('mnist', train=True, 
									  download=True, transform=transforms.ToTensor())
testset = torchvision.datasets.MNIST('mnist', train=False, 
									  download=True, transform=transforms.ToTensor())

# Prepare training loader and testing loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
shuffle=False, num_workers=0) 

In [11]:
# Define the class Net
class Net(nn.Module):
    def __init__(self):    
    	# Define all the parameters of the net
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28 * 1, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):    
    	# Do the forward pass
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
# Instantiate the Adam optimizer and Cross-Entropy loss function
model = Net()   
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
  
for batch_idx, data_target in enumerate(trainloader):
    data = data_target[0]
    target = data_target[1]
    data = data.view(-1, 28 * 28)
    optimizer.zero_grad()

    # Complete a forward pass
    output = model(data)

    # Compute the loss, gradients and change the weights
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

In [14]:
# Set the model in eval mode
model.eval()
total =0
correct=0
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    
    # Put each image into a vector
    inputs = inputs.view(-1, 28 * 28)
    
    # Do the forward pass and get the predictions
    outputs = model(inputs)
    _, outputs = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (outputs == labels).sum().item()
print('The testing set accuracy of the network is: %d %%' % (100 * correct / total))

The testing set accuracy of the network is: 93 %


In [20]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Instantiate two convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=10, kernel_size=3, padding=1)
        
        # Instantiate the ReLU nonlinearity
        self.relu = nn.ReLU(inplace=True)
        
        # Instantiate a max pooling layer
        self.pool = nn.MaxPool2d(2,2)
        
        # Instantiate a fully connected layer
        self.fc = nn.Linear(7*7*10, 10)
    def forward(self, x):
  
        # Apply conv followd by relu, then in next line pool
        x = self.relu(self.conv1(x))
        x = self.pool(x)

        # Apply conv followd by relu, then in next line pool
        x = self.relu(self.conv2(x))
        x = self.pool(x)

        # Prepare the image for the fully connected layer
        x = x.view(-1, 7 * 7 * 10)

        # Apply the fully connected layer and return the result
        return self.fc(x)

In [25]:
# modeli ekle, loss ve optimizer ekle 
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#modeli eğit
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0


[1,   200] loss: 1.162
[1,   400] loss: 0.378
[1,   600] loss: 0.252
[1,   800] loss: 0.210
[1,  1000] loss: 0.184
[1,  1200] loss: 0.161
[1,  1400] loss: 0.140
[1,  1600] loss: 0.125
[1,  1800] loss: 0.116
[2,   200] loss: 0.105
[2,   400] loss: 0.097
[2,   600] loss: 0.096
[2,   800] loss: 0.096
[2,  1000] loss: 0.088
[2,  1200] loss: 0.089
[2,  1400] loss: 0.097
[2,  1600] loss: 0.099
[2,  1800] loss: 0.095
[3,   200] loss: 0.080
[3,   400] loss: 0.082
[3,   600] loss: 0.076
[3,   800] loss: 0.078
[3,  1000] loss: 0.082
[3,  1200] loss: 0.078
[3,  1400] loss: 0.072
[3,  1600] loss: 0.071
[3,  1800] loss: 0.076
[4,   200] loss: 0.068
[4,   400] loss: 0.064
[4,   600] loss: 0.067
[4,   800] loss: 0.077
[4,  1000] loss: 0.065
[4,  1200] loss: 0.062
[4,  1400] loss: 0.060
[4,  1600] loss: 0.070
[4,  1800] loss: 0.069
[5,   200] loss: 0.069
[5,   400] loss: 0.055
[5,   600] loss: 0.056
[5,   800] loss: 0.064
[5,  1000] loss: 0.065
[5,  1200] loss: 0.066
[5,  1400] loss: 0.056
[5,  1600] 

In [27]:
# model.eval() dediğimizde dropout katmanı çalışmaz.
model.eval()
correct_total = 0
total = 0
correct_class = [0] * 10
total_class = [0] * 10

with torch.no_grad():
    for inputs, labels in testloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct_total += (predicted == labels).sum().item()
        for i in range(labels.size(0)):
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                correct_class[label] += 1
            total_class[label] += 1

accuracy_total = 100 * correct_total / total
print('Accuracy of the network on the test set: %d %%' % accuracy_total)

for i in range(10):
    accuracy_class = 100 * correct_class[i] / total_class[i]
    print('Accuracy of %5s : %2d %%' % (
        testset.classes[i], accuracy_class))

Accuracy of the network on the test set: 98 %
Accuracy of 0 - zero : 99 %
Accuracy of 1 - one : 99 %
Accuracy of 2 - two : 97 %
Accuracy of 3 - three : 96 %
Accuracy of 4 - four : 99 %
Accuracy of 5 - five : 98 %
Accuracy of 6 - six : 97 %
Accuracy of 7 - seven : 98 %
Accuracy of 8 - eight : 98 %
Accuracy of 9 - nine : 96 %
